# **PREDICCIONES PARA UN SOLO PARTICIPANTE**

In [1]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sbn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import random as random
import plotly.express as px
import tensorflow as tf
from tensorflow.keras import layers

def set_output_precision(decimals):
  """
  format the output of the all the data structures
  with an specific number of decimals
  """
  np.set_printoptions(precision=decimals)
  into='{'+':.{}f'.format(decimals)+'}'
  pd.options.display.float_format = into.format
  torch.set_printoptions(precision=decimals)
  pass

def plot_ts(df,dfx="Minute",dfy="METS",_title="DF minute x Mets"):
  if not isinstance(df,pd.DataFrame):
    df = pd.DataFrame({'METS': df, 'Minute': range(len(df))})

  plt.figure()
  fig = px.line(df, x = dfx, y = dfy , title = _title)
  fig.update_xaxes(
      rangeslider_visible = True,
      rangeselector = dict(
          buttons = list([
              dict(count=1,label="1y",step="year",stepmode="backward"),
              dict(count=2,label="2y",step="year",stepmode="backward"),
              dict(count=3,label="3y",step="year",stepmode="backward"),
              dict(step="all")
          ])
      )

  )
  fig.show()

def plot_predictions_vs_real(predictions, reals):
  df = pd.DataFrame()
  number_of_points = len(predictions)
  df["time"] = range(0,number_of_points)
  df["participant"] = "prediction"
  df["value"] = predictions
  for i in range(0,number_of_points):
    df.loc[number_of_points+i] = [i,"real",reals[i]]

  plt.figure(1)
  fig = px.line(df, x = "time", y = "value" , title = "predictions vs reals" , color = "participant")
  fig.update_xaxes(
        rangeslider_visible = True,
        rangeselector = dict(
            buttons = list([
                dict(count=1,label="1y",step="year",stepmode="backward"),
                dict(count=2,label="2y",step="year",stepmode="backward"),
                dict(count=3,label="3y",step="year",stepmode="backward"),
                dict(step="all")
            ])
        )

    )
  fig.show()

In [2]:
READ_LOCAL_DATA = True
COMPUTED_OPTION = 2
TEST_SIZE = 0.2
VALIDATION_SIZE = 0.15
SAVE_RESULTS = True
LOW_DATA = True

In [3]:
if READ_LOCAL_DATA:
  PATH = "Resources/Individual/"
else:
    #  We start by getting access to the drive
    from google.colab import drive
    drive.mount('/content/drive')
    PATH = "/content/drive/MyDrive/TFG/Resources/Individual/"

if LOW_DATA:
    PATH += "LowData/"

In [4]:
import gzip
documents = ['minuteY','hourY','dayY']
# with open(PATH+"minuteX"+".pkl", 'rb') as file:
#     dataX = np.array(pickle.load(file),np.float32)
file = PATH+"minuteX.pkl.gz"
dataX = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)

file = PATH+documents[COMPUTED_OPTION]+".pkl.gz"
dataY = np.array(pickle.load(gzip.open(file, 'rb')),np.float32)


In [5]:
NUMBER_OF_PARTICIPANTS = dataX[0]
print(dataX.shape)
print(file)

# Calcular la variabilidad de cada elemento en X
variability = np.std(dataX, axis=(1, 2))  # Calcular la desviación estándar

# Obtener los índices de los 25 elementos con mayor variabilidad
top_indices = np.argsort(variability)[-25:]

# Extraer los mismos elementos de X y Y utilizando los índices
selected_dataX = dataX[top_indices, :, :]
print()

(25, 36355, 1440)
Resources/Individual/LowData/dayY.pkl.gz



In [6]:
def change_shape_by_participant(data):
    original_shape = data.shape
    new_shape = (original_shape[0] * original_shape[1], original_shape[2])
    reshaped_array = data.reshape(new_shape)
    return reshaped_array

In [7]:
dataX = dataX.transpose(1,0,2)
dataY = dataY.transpose(1,0,2)
print(dataX.shape)
print(dataY.shape)

(36355, 25, 1440)
(36355, 25, 1)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(dataX, dataY, test_size=TEST_SIZE,shuffle=True,random_state=42)
del dataY
del dataX
X_train = X_train.transpose(1,0,2)
X_test = X_test.transpose(1,0,2)
y_train = y_train.transpose(1,0,2)
y_test =  y_test.transpose(1,0,2)

In [9]:
X_train = change_shape_by_participant(X_train)
X_test = change_shape_by_participant(X_test)
y_train = change_shape_by_participant(y_train)
y_test = change_shape_by_participant(y_test)

In [10]:
print("Examples for training\n","X:",X_train.shape,"y:",y_train.shape)
print("Examples for test\n","X:",X_test.shape,"y:",y_test.shape)
VALIDATION_INDEX = int(len(X_train)*VALIDATION_SIZE)
print("Examples for validation: \n", VALIDATION_INDEX)

Examples for training
 X: (727100, 1440) y: (727100, 1)
Examples for test
 X: (181775, 1440) y: (181775, 1)
Examples for validation: 
 109065


In [11]:
tf.random.set_seed(42)
# Setup dataset hyperparameters
HORIZON = y_test.shape[1]
WINDOW_SIZE = 1440

# Let's build an LSTM model with the Functional API
inputs = layers.Input(shape=(WINDOW_SIZE))
x = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))(inputs) # expand input dimension to be compatible with LSTM
# print(x.shape)
# x = layers.LSTM(128, activation="relu", return_sequences=True)(inputs) # this layer will error if the inputs are not the right shape
x = layers.LSTM(128,return_sequences=True, activation="relu")(x) # using the tanh loss function results in a massive error
# print(x.shape)
# Add another optional dense layer (you could add more of these to see if they improve model performance)
# x = layers.Dense(32, activation="relu")(x)
output = layers.Dense(HORIZON)(x)
model_LSTM = tf.keras.Model(inputs=inputs, outputs=output, name="model_5_lstm")

model_LSTM.summary()

Model: "model_5_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1440)]            0         
                                                                 
 lambda (Lambda)             (None, 1, 1440)           0         
                                                                 
 lstm (LSTM)                 (None, 1, 128)            803328    
                                                                 
 dense (Dense)               (None, 1, 1)              129       
                                                                 
Total params: 803,457
Trainable params: 803,457
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile model
model_LSTM.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(),
             metrics=["mae"])

In [13]:
# Seems when saving the model several warnings are appearing: https://github.com/tensorflow/tensorflow/issues/47554
model_LSTM.fit(X_train[:-VALIDATION_INDEX],
            y_train[:-VALIDATION_INDEX],
            epochs=3,
            verbose=1,
            batch_size=128,
            shuffle=True,
            validation_data=(X_train[-VALIDATION_INDEX:], y_train[-VALIDATION_INDEX:]))

Epoch 1/3
4829/4829 [==============================] - 175s 35ms/step - loss: 240.1146 - mae: 240.1146 - val_loss: 426.8304 - val_mae: 426.8304
Epoch 2/3
4829/4829 [==============================] - 135s 28ms/step - loss: 206.8793 - mae: 206.8793 - val_loss: 406.4524 - val_mae: 406.4524
Epoch 3/3
4829/4829 [==============================] - 73s 15ms/step - loss: 203.9804 - mae: 203.9804 - val_loss: 401.1087 - val_mae: 401.1087


In [14]:
def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model 
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data)
  return tf.squeeze(forecast) # return 1D array of predictions

predictions = make_preds(model_LSTM, X_test)

5681/5681 [==============================] - 23s 4ms/step


In [15]:
print(predictions.shape)
print(y_test.shape)
print('\033[1m' + "MSE: " + str(mean_squared_error(y_test,predictions)) + '\033[0m')
print('\033[1m' + "MAE: " + str(mean_absolute_error(y_test,predictions)) + '\033[0m')
print(predictions[0].shape)

if COMPUTED_OPTION < 2:
    y_test_suma = np.sum(y_test,axis=1)
    predictions_suma = np.sum(predictions,axis=1)
    print(y_test_suma.shape)
    print(predictions_suma.shape)
    print('\033[1m' + "MSE: " + str(mean_squared_error(y_test_suma,predictions_suma)) + '\033[0m')
    print('\033[1m' + "MAE: " + str(mean_absolute_error(y_test_suma,predictions_suma)) + '\033[0m')

(181775,)
(181775, 1)
MSE: 215561.33
MAE: 231.14833
()


In [16]:
if COMPUTED_OPTION < 2:
    plot_predictions_vs_real(predictions=predictions[1000],reals=y_test[1000])
else:
    print(predictions[0],y_test[0])
    print()

tf.Tensor(2065.6702, shape=(), dtype=float32) [2191.2551]



In [17]:
if SAVE_RESULTS:
    file_path = 'Resources/Resultados/Individual/'
    if LOW_DATA:
        file_path += "LowData/"
    documents = ['minuteY','hourY','dayY']
    file = file_path+documents[COMPUTED_OPTION]+"-predictions"+".pkl.gz"
    pickle.dump(predictions, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-test"+".pkl.gz"
    pickle.dump(y_test, gzip.open(file, 'wb'))
    file = file_path+documents[COMPUTED_OPTION]+"-X"+".pkl.gz"
    pickle.dump(X_test, gzip.open(file, 'wb'))